In [7]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Button
from bokeh.io import output_notebook, push_notebook
from PIL import Image
from io import BytesIO
import requests
from astropy.io import fits

# Enable Bokeh output in the notebook
output_notebook()

# Function to load image (you can replace this with a local file if preferred)
def load_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return np.array(img)

# Load the galaxy image (replace with your image URL)
img = fits.getdata('/Users/user/Documents/PhD/proposals/2573_F356Wim.fits')

# Create ColumnDataSource for selected pixels
source = ColumnDataSource(data=dict(x=[], y=[]))

# Create the main plot
p = figure(x_range=(0, img.shape[1]), y_range=(0, img.shape[0]),
           width=800, height=600, title="Galaxy Pixel Selector",
           tools="box_select,lasso_select,reset")

# Display the image
p.image_rgba(image=[img], x=0, y=0, dw=img.shape[1], dh=img.shape[0])

# Add box select and lasso select tools


# Function to update selected pixels
def update_selection(attr, old, new):
    selected = source.selected.indices
    x, y = np.unravel_index(selected, (img.shape[0], img.shape[1]))
    source.data = dict(x=x, y=y)
    push_notebook()

# Connect the selection to the update function
source.selected.on_change('indices', update_selection)

# Function to save coordinates
def save_coordinates():
    coordinates = np.column_stack((source.data['x'], source.data['y']))
    np.savetxt("selected_pixels.txt", coordinates, fmt='%d')
    print("Coordinates saved to selected_pixels.txt")

# Create a button to save coordinates
save_button = Button(label="Save Coordinates", button_type="success")
save_button.on_click(save_coordinates)

# Create the layout
layout = column(p, save_button)

# Show the plot
show(layout, notebook_handle=True)

Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

